In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [10]:
import torch.optim as optim
from torch.utils.data import DataLoader
import sys
sys.path.append('..')
from shared.models import *
from shared.datasets import *

def train(model, train_loader, criterion, device, optimizer):
    model.train()
    train_loss = 0
    for step, (data_inputs, data_labels) in enumerate(train_loader):
        inputs, labels = data_inputs.to(device), data_labels.to(device) # Convert Tensors to appropriate device
        optimizer.zero_grad() 
        pred = model(inputs)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() # Running training loss
        
    return train_loss/step

def test(model, test_loader, criterion, device, n_way):
    model.eval()
    true_positive = list(0. for i in range(n_way)) # Number of correctly predicted samples per class
    total_truth = list(0. for i in range(n_way)) # Number of ground truths per class
    predicted_positive = list(0. for i in range(n_way)) # Number of predicted samples per class
    precision = list(0. for i in range(n_way))
    recall = list(0. for i in range(n_way))
    val_loss = 0
    correct_total = 0 # Total correctly predicted samples
    total = 0 # Total samples
    f1_flag = 0 # Flag for if the model does not predict any positives for a class which breaks precision and F1 score
    with torch.no_grad():
        for step, (data_inputs, data_labels) in enumerate(test_loader):
            inputs, labels = data_inputs.to(device), data_labels.to(device)
            pred = model(inputs)
            loss = criterion(pred, labels)
            val_loss += loss.item() # Running validation loss
            _, predicted = torch.max(pred, 1)
            correct = (predicted == labels).squeeze() # Samples that are correctly predicted
            correct_total += (predicted == labels).sum().item()
            total += labels.size(0)
            
            for i in range(len(predicted)):
                label = labels[i]
                true_positive[label] += correct[i].item() 
                total_truth[label] += 1
                predicted_positive[predicted[i].item()] += 1 # True Positive + False Positive
                
        # Find class accuracy, precision and recall
        for j in range(n_way):
            if (predicted_positive[j] == 0 or true_positive[j] == 0):
                f1_flag = 1
            else:
                precision[j] = true_positive[j] / predicted_positive[j]
            recall[j] = true_positive[j] / total_truth[j] # Recall is the same as per class accuracy
            
        # Find Accuracy, Macro Accuracy and Macro F1 Score
        macro_acc_sum = 0
        f1_sum = 0
        for k in range(n_way):
            macro_acc_sum += recall[k]
            if f1_flag == 0: # Check for broken f1 score
                f1_sum = 2 * precision[k] * recall[k] / (precision[k] + recall[k])
                
        accuracy = correct_total/total
        macro_accuracy = macro_acc_sum/n_way 
        if f1_flag == 0: 
            f1_score = f1_sum/n_way
        else:
            f1_score = -1
        
    return val_loss/step, accuracy, macro_accuracy, f1_score

if __name__ == '__main__':

    num_epochs = 30
    num_workers = 8
    bs = 64
    n_way = 10
    
    torch.cuda.set_device(0)
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = BaselineNet(n_way).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.0001)

    train_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                                csv_path='./splits.csv', mode='base_train', resize=224)
    test_dataset = MimicCxrJpg(root='../../../../scratch/rl80/mimic-cxr-jpg-2.0.0.physionet.org/files/',
                                csv_path='./splits.csv', mode='base_validate', resize=224)
    train_loader = DataLoader(train_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)
    test_loader = DataLoader(test_dataset, batch_size=bs, shuffle=True, num_workers=num_workers)

    for epoch in range(num_epochs):
        train_loss = train(model, train_loader, criterion, device, optimizer)
        val_loss, acc, m_acc, f1 = test(model, test_loader, criterion, device, n_way)

        print(f'[{epoch+1}] t_loss: {train_loss:.5f} v_loss: {val_loss:.5f} val_acc: {acc:.5f} val_m_acc: {m_acc:.5f} f1: {f1:.5f}')

[1] t_loss: 1.43548 v_loss: 1.17643 val_acc: 0.32444 val_m_acc: 0.32444 f1: 0.00442
[2] t_loss: 1.29332 v_loss: 1.19555 val_acc: 0.33333 val_m_acc: 0.33333 f1: -1.00000
[3] t_loss: 1.11859 v_loss: 1.19735 val_acc: 0.37667 val_m_acc: 0.37667 f1: -1.00000
[4] t_loss: 1.01132 v_loss: 1.24339 val_acc: 0.38222 val_m_acc: 0.38222 f1: -1.00000
[5] t_loss: 0.89777 v_loss: 1.24683 val_acc: 0.34556 val_m_acc: 0.34556 f1: -1.00000
[6] t_loss: 0.82048 v_loss: 1.26053 val_acc: 0.41333 val_m_acc: 0.41333 f1: -1.00000
[7] t_loss: 0.75447 v_loss: 1.27558 val_acc: 0.41667 val_m_acc: 0.41667 f1: -1.00000
[8] t_loss: 0.69568 v_loss: 1.25815 val_acc: 0.42778 val_m_acc: 0.42778 f1: -1.00000
[9] t_loss: 0.64085 v_loss: 1.27643 val_acc: 0.38333 val_m_acc: 0.38333 f1: 0.11315
[10] t_loss: 0.59549 v_loss: 1.20504 val_acc: 0.42111 val_m_acc: 0.42111 f1: 0.16882
[11] t_loss: 0.54603 v_loss: 1.20704 val_acc: 0.42222 val_m_acc: 0.42222 f1: 0.15926
[12] t_loss: 0.49984 v_loss: 1.21013 val_acc: 0.43667 val_m_acc: 0.

In [1]:
def train(model, train_loader, criterion, device, optimizer):
    model.train()
    train_loss = 0
    for step, (data_inputs, data_labels) in enumerate(train_loader):
        inputs, labels = data_inputs.to(device), data_labels.to(device) # Convert Tensors to appropriate device
        optimizer.zero_grad() 
        pred = model(inputs)
        loss = criterion(pred, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() # Running training loss
        
    return train_loss/step

def test(model, test_loader, criterion, device, n_way):
    model.eval()
    true_positive = list(0. for i in range(n_way)) # Number of correctly predicted samples per class
    total_truth = list(0. for i in range(n_way)) # Number of ground truths per class
    predicted_positive = list(0. for i in range(n_way)) # Number of predicted samples per class
    precision = list(0. for i in range(n_way))
    recall = list(0. for i in range(n_way))
    val_loss = 0
    correct_total = 0 # Total correctly predicted samples
    total = 0 # Total samples
    with torch.no_grad():
        for step, (data_inputs, data_labels) in enumerate(test_loader):
            inputs, labels = data_inputs.to(device), data_labels.to(device)
            pred = model(inputs)
            loss = criterion(pred, labels)
            val_loss += loss.item() # Running validation loss
            _, predicted = torch.max(pred, 1)
            correct = (predicted == labels).squeeze() # Samples that are correctly predicted
            correct_total += (predicted == labels).sum().item()
            total += labels.size(0)
            
            for i in range(len(predicted)):
                label = labels[i]
                true_positive[label] += correct[i].item() 
                total_truth[label] += 1
                predicted_positive[predicted[i].item()] += 1 # True Positive + False Positive
                
        # Find class accuracy, precision and recall
        for j in range(n_way):
            precision[j] = true_positive[j] / predicted_positive[j]
            recall[j] = true_positive[j] / total_truth[j] # Recall is the same as per class accuracy
            
        # Find Accuracy, Macro Accuracy and Macro F1 Score
        macro_acc_sum = 0
        f1_sum = 0
        for k in range(n_way):
            macro_acc_sum += recall[k]
            f1_sum = 2 * precision[k] * recall[k] / (precision[k] + recall[k])
        accuracy = correct_total/total
        macro_accuracy = macro_acc_sum/n_way 
        f1_score = f1_sum/n_way
        
    return val_loss/step, accuracy, macro_accuracy, f1_score

SyntaxError: invalid syntax (<ipython-input-1-a6c726f51a75>, line 1)

In [8]:
precision[k]

ValueError: Precision must be int or format string, not '[k]'

In [11]:
test = {'a': 1}

In [13]:
test['a']

1

In [28]:
correct = (predicted == labels).squeeze()
for i in range(len(predicted)):
    label = labels[i]
    class_correct[label] += correct[i].item()
    class_total[label] += 1

In [31]:
class_correct / class_total

TypeError: unsupported operand type(s) for /: 'list' and 'list'

In [37]:
total = 0
for j in range(n_way):
    total += class_correct[j] / class_total[j]
accuracy = total/n_way

0.0
0.6666666666666666
0.0
0.3333333333333333
0.0
0.2222222222222222
0.14285714285714285
0.0
0.0
0.5
0.1865079365079365


In [35]:
test = 0
test += 1.5
test

1.5

In [21]:
c

tensor([False,  True, False, False, False, False, False, False, False, False,
        False,  True,  True, False, False, False, False, False, False,  True,
        False,  True, False, False,  True, False, False, False,  True, False,
        False,  True, False, False, False], device='cuda:0')